In [ ]:
import pathlib as pl
import datetime as dt

import shapely
import pandas as pd
import r5py as r5
import pyrosm as pr
import geopandas as gpd
import geohexgrid as ghg
from loguru import logger

gpd.options.io_engine = "pyogrio"


# Setup

In [ ]:
DATA_DIR = pl.Path("../data") 
WGS84 = "epsg:4326"
NZTM = "epsg:2193"

%ls {DATA_DIR}


# Write an isochrone function using r5py

To help address [this Github issue](https://github.com/r5py/r5py/issues/311) .
Actually, write two such functions, one based on a user-specified set of grid cells and
one based on the [concave hull isochrones of r5r](https://github.com/ipeaGIT/r5r/blob/master/r-package/R/isochrone.R).

I find the grid-style function more useful for my work (which often involves gridding up a study area for sampling),
because (1) i can set the resolution of my isochrones (via the grid), 
and (2) i needn't iterate to find a perfect convex hull ratio to use for accurate-looking results.


In [ ]:
def isochrone_g(
    transport_network: r5.TransportNetwork,
    transport_modes: list[r5.TransportMode],
    origins: gpd.GeoDataFrame,
    time_bounds: list[float],
    grid: gpd.GeoDataFrame,
    destinations: gpd.GeoDataFrame | None = None,
    departure: dt.datetime | None = None,
    snap_to_network: bool | int = False,
    **kwargs: dict,
) -> gpd.GeoDataFrame:
    """
    Compute grid-style isochrones from the given origin points.

    More specifically, given a GeoDataFrame of WGS84 origin points with a unique
    identifier column 'id' and a grid of WGS84 polygons with unique identifier column
    'id', whose polygons ideally cover the study area without overlaps 
    and are clipped to coastlines, do the following.
    Choose one representative point from each grid cell to form a set of destination points.
    Alternatively, use ``destinations``, if given, which should also have an 'id' column
    that matches the grid cell IDs.
    For each origin and for each duration t (minutes) in the given list of time bounds,
    find all destinations that are at most t minutes away when departing from the
    origin at the given departure time (which defaults to the current datetime),
    using the given transport modes, and travelling through the given transport nework.
    Collect all grid cells containing those reachable destinations, dissolve them,
    and set that as the isochrone for that origin and duration.
    Return a GeoDataFrame with the following columns.

    - ``'from_id'``: origin point ID
    - ``'travel_time_percentile'``: string; defaluts to 'p50' if no percentiles are given
    - ``'time_bound'``: float; one of the given time bounds
    - ``'geometry'``: (Multi)Polygon isochrone.

    You can customise the isochrone calculation as follows.

    - Snap the origin points to the street network before routing if and only if
      ``snap_to_network``
      If ``True``, then the default search radius
      defined in com.conveyal.r5.streets.StreetLayer.LINK_RADIUS_METERS is used;
      if int, then use that many meters as the search radius for snapping.
    - Pass in any keyword arguments accepted by :class:`r5py.RegionalTask`,
      e.g. `departure_time_window`, `percentiles`, `max_time_walking`.

    NOTES:

    - All given GeoDataFrame will be converted to coordinate reference system (CRS) WGS84 before routing, 
      then converted back to the CRS of the grid.
    - Uses r5py to do the routing.

    """
    # Prepare inputs
    logger.info("Prepare inputs")
    time_bounds = sorted(set(time_bounds))
    origins = origins.to_crs(WGS84)
    final_crs = grid.crs
    grid = grid.to_crs(WGS84)
    if destinations is None:
        destinations = grid.assign(geometry=lambda x: x.representative_point())
    else:
        destinations = destinations.to_crs(WGS84)

    # Compute travel times
    logger.info("Compute travel times")
    ttm = r5.TravelTimeMatrixComputer(
        transport_network,
        origins=origins,
        destinations=destinations,
        departure=departure,
        transport_modes=transport_modes,
        snap_to_network=snap_to_network,
        **kwargs,
    )
    f = (
        ttm.compute_travel_times()
        .dropna()
        .rename(columns={"travel_time": "travel_time_p50"})
        # Melt in case of multiple travel time percentiles
        .melt(id_vars=["from_id", "to_id"], var_name="pctile", value_name="travel_time")
        .assign(pctile=lambda x: x["pctile"].str.split("_").str[-1])
    )
    if f.empty:
        return gpd.GeoDataFrame()

    # Build isochrones from the grid cells of the reachable points
    logger.info("Build isochrones")
    records = []
    for (from_id, pctile), group in f.groupby(["from_id", "pctile"]):
        for time_bound in time_bounds:
            iso = grid.merge(
                group.loc[lambda x: x["travel_time"] <= time_bound].rename(
                    columns={"to_id": "id"}
                )
            ).dissolve()
            records.append(
                {
                    "from_id": from_id,
                    "travel_time_percentile": pctile,
                    "time_bound": time_bound,
                    "geometry": iso["geometry"].iat[0] if not iso.empty else np.nan,
                }
            )

    return gpd.GeoDataFrame(pd.DataFrame.from_records(records), crs=WGS84).to_crs(final_crs)

def get_osm_nodes(transport_network: r5.TransportNetwork) -> gpd.GeoDataFrame:
    """
    Return the OSM nodes underlying the given transport network.
    Include in the GeoDataFrame an ID column 'id' that is simply the index of the 
    GeoDataFrame.
    """
    import com.conveyal.r5
    
    k = com.conveyal.r5.streets.VertexStore.FIXED_FACTOR
    v = transport_network._transport_network.streetLayer.vertexStore
    lonlats = zip(list(v.fixedLons.toArray()), list(v.fixedLats.toArray()))
    nodes = gpd.GeoDataFrame(
        geometry=[shapely.Point(lon / k, lat / k) for lon, lat in lonlats],
        crs="epsg:4326",
    )
    nodes["id"] = nodes.index
    return nodes

def isochrone_ch(
    transport_network: r5.TransportNetwork,
    transport_modes: list[r5.TransportMode],
    origins: gpd.GeoDataFrame,
    time_bounds: list[float],
    destinations: gpd.GeoDataFrame | None = None,
    departure: dt.datetime|None=None,
    snap_to_network: bool|int=False,
    sample_frac: float=0.8,
    concave_hull_ratio=0.15,
    **kwargs: dict,
) -> gpd.GeoDataFrame:
    """
    Compute concave-hull-style isochrones from the given origin points.

    More specifically, given a GeoDataFrame of WGS84 origin points with a unique
    identifier column 'id' do the following.
    Choose a set of destination points by sampling ``sample_frac`` of all OSM nodes in
    the underlying network.
    Alternatively, use ``destinations``, if given, which should also have an 'id' column
    that matches the grid. 
    For each origin and for each duration t (minutes) in the given list of time bounds,
    find all destinations that are at most t minutes away when departing from the
    origin at the given departure time (which defaults to the current datetime),
    using the given transport modes, and travelling through the given transport nework.
    Collect all such destinations, compute their concave hull using the given
    concave hull ratio, and set that as the isochrone for that origin and duration.
    Return a GeoDataFrame with the following columns.

    - ``'from_id'``: origin point ID
    - ``'travel_time_percentile'``: string; defaluts to 'p50' if no percentiles are given
    - ``'time_bound'``: float; one of the given time bounds
    - ``'geometry'``: (Multi)Polygon isochrone.

    You can customise the isochrone calculation as follows.

    - Snap the origin points to the street network before routing if and only if
      ``snap_to_network``
      If ``True``, then the default search radius
      defined in com.conveyal.r5.streets.StreetLayer.LINK_RADIUS_METERS is used;
      if int, then use that many meters as the search radius for snapping.
    - Pass in any keyword arguments accepted by :class:`r5py.RegionalTask`,
      e.g. `departure_time_window`, `percentiles`, `max_time_walking`.

    NOTES:

    - All given GeoDataFrames will be converted to coordinate reference system (CRS) WGS84 before routing, 
      then converted back to the CRS of the origin points.
    - Uses r5py to do the routing.

    """
    logger.info("Prepare inputs")
    final_crs = origins.crs
    time_bounds = sorted(set(time_bounds))
    osm_nodes = get_osm_nodes(transport_network).sample(frac=sample_frac, random_state=1)

    # Compute travel times
    logger.info("Compute travel times")
    ttm = r5.TravelTimeMatrixComputer(
        transport_network,
        origins=origins.to_crs(WGS84),
        destinations=osm_nodes,
        departure=departure,
        transport_modes=transport_modes,
        snap_to_network=snap_to_network,
        **kwargs,
    )
    f = (
        ttm.compute_travel_times()
        .dropna()
        .rename(columns={"travel_time": "travel_time_p50"})
        # Melt in case of multiple travel time percentiles
        .melt(id_vars=["from_id", "to_id"], var_name="pctile", value_name="travel_time")
        .assign(pctile=lambda x: x["pctile"].str.split("_").str[-1])
    )
    if f.empty:
        return gpd.GeoDataFrame()

    # Build isochrones as concave hulls of reachable points
    logger.info("Build isochrones")
    records = []
    for (from_id, pctile), group in f.groupby(["from_id", "pctile"]):
        for time_bound in time_bounds:
            reachable_nodes = osm_nodes.merge(
                group
                .loc[lambda x: x["travel_time"] <= time_bound]
                .rename(columns={"to_id": "id"})
            )
            iso = shapely.concave_hull(reachable_nodes.unary_union, ratio=concave_hull_ratio)
            records.append(
                {
                    "from_id": from_id,
                    "travel_time_percentile": pctile,
                    "time_bound": time_bound,
                    "geometry": iso if not iso.is_empty else np.nan,
                }
            )

    logger.info("Build GeoDataFrame")
    return gpd.GeoDataFrame(pd.DataFrame.from_records(records), crs=WGS84).to_crs(final_crs)
  


# Illustrate the functions 

Use data from my location of Auckland, New Zealand, which i can confidently scrutinise.

In [ ]:
# Load Auckland OSM and GTFS data
%time akl_pbf_path = pr.get_data("Auckland", directory=DATA_DIR)
akl_gtfs_path = DATA_DIR / "auckland_gtfs_20230824.zip"

# Make the R5 transport network from these
%time transport_network = r5.TransportNetwork(akl_pbf_path, [akl_gtfs_path])
transport_modes = [
    r5.TransportMode.TRANSIT,
    r5.TransportMode.WALK,
]

In [ ]:
%%time

# Make a hexagon grid of circumradius 100 m covering the transport network
# and clipping to coastlines
study_area = (
    # Contains coastlines
    gpd.read_file(DATA_DIR / "auckland.gpkg")  
    .clip(
        gpd.GeoDataFrame(geometry=[transport_network.extent], crs=WGS84)
        .to_crs(NZTM)
    )
)
path = DATA_DIR / "auckland_grid_100m.gpkg"
if not path.exists():
    logger.info("Making grid")
    grid = ghg.make_grid_from_gdf(study_area, 100, clip=True)
    grid.to_file(path, driver="GPKG")
else:
    grid = gpd.read_file(path)

display(grid.head())
print("#grid cells =", grid.shape[0])

# Plot the study area and the some grid cells.
# The entire grid takes too long to plot
pd.concat([study_area, grid.iloc[55_000:56_000]]).explore(tiles="CartoDB positron")


In [ ]:
# Get some origin points, some of which might involve ferry rides

origins = gpd.read_file(DATA_DIR / "auckland_points.geojson").assign(id=lambda x: x.index).to_crs(NZTM)
display(origins)
display(origins.assign(geometry=lambda x: x.buffer(50)).explore(tiles="CartoDB positron"))


In [ ]:
%%time

# Takes ~15 seconds on my computer
isos = (
    isochrone_g(
        transport_network=transport_network, 
        transport_modes=transport_modes,
        origins=origins,
        departure=dt.datetime(2023, 8, 28, 8, 0, 0),
        time_bounds=[45],
        grid=grid.rename(columns={"cell_id": "id"}),
        departure_time_window=dt.timedelta(seconds=35*60),
        percentiles=[1, 25, 50],
    )
)

display(isos.head())
for ptile, group in isos.groupby("travel_time_percentile"):
    print(ptile)
    display(group.explore(column="from_id", categorical=True, cmap="viridis", tiles="CartoDB positron"))

In [ ]:
%%time

# Takes ~77 seconds on my computer
isos = (
    isochrone_ch(
        transport_network=transport_network, 
        transport_modes=transport_modes,
        origins=origins,
        departure=dt.datetime(2023, 8, 28, 8, 0, 0),
        time_bounds=[45],
        departure_time_window=dt.timedelta(seconds=35*60),
        percentiles=[1, 25, 50],
    )
)

display(isos.head())
for ptile, group in isos.groupby("travel_time_percentile"):
    print(ptile)
    display(group.explore(column="from_id", categorical=True, cmap="viridis", tiles="CartoDB positron"))